In [2]:
import pandas as pd
import matplotlib.pyplot as plt
excel_file = "/Users/nargeshosseinzadeh/Downloads/Data from Database.xlsx"
data = pd.read_excel(excel_file)

Separate plots for each each vital sign, per patient (Temperature is blue, HR is Orange, SBP is Brown and DBP is green for all patients):

In [ ]:
# Group data by 'Patient_ID'
grouped_data = data.groupby('patient_id')

# Get a colormap with a unique color for each patient
num_patients = len(grouped_data)
colors = plt.cm.get_cmap('tab10', num_patients)  # 'tab10' colormap provides 10 distinct colors

# Define vital signs
vital_signs = ['T', 'HR', 'SBP', 'DBP']

# Iterate through each patient group
for patient_id, patient_data in grouped_data:
    # Parse DateTime column as datetime
    patient_data['obs_ts'] = pd.to_datetime(patient_data['obs_ts'])

    # Create individual plots for each vital sign
    for idx, vital_sign in enumerate(vital_signs):
        # Create a new plot for each vital sign
        plt.figure(figsize=(8, 6))
        
        # Plot with a unique color
        color = colors(idx / len(vital_signs))  # Get a color from the colormap
        plt.plot(patient_data['obs_ts'], patient_data[vital_sign], marker='o', linestyle='-', color=color)
        
        plt.xlabel('Date and Time')
        plt.ylabel(vital_sign)
        plt.title(f'Patient {patient_id} - {vital_sign}')
        plt.tight_layout()
        plt.show()


Let's define the following thresholds:

In [ ]:

# Group data by 'Patient_ID'
grouped_data = data.groupby('patient_id')

# Threshold values (using tuples for threshold ranges)
thresholds = {
    'T': (36, 38),  # Temperature: 36 to 38 degree Celsius
    'HR': (90, float('inf')),  # Heart Rate: > 90 beats/min
    'SBP': (float('-inf'), 100),  # Systolic Blood Pressure: <= 100 mm Hg
    'R': (20, float('inf'))  # Respiratory Rate: > 20 beats/min
}

# Define vital signs and their colors
vital_signs = ['T', 'HR', 'SBP', 'R']
colors = {'T': 'blue', 'HR': 'black', 'SBP': 'green', 'R': 'orange'}

# Iterate through each patient group
for patient_id, patient_data in grouped_data:
    # Parse DateTime column as datetime
    patient_data['obs_ts'] = pd.to_datetime(patient_data['obs_ts'])
    
    # Create a 4-row grid for subplots
    fig, axs = plt.subplots(4, 1, figsize=(8, 10), sharex=True)  # sharex=True ensures the x-axis is shared
    
    # Plot each vital sign in a separate subplot
    for i, vital_sign in enumerate(vital_signs):
        ax = axs[i]  # Get the current subplot
        
        # Plot vital sign data with different colors based on the specified colors dictionary
        vital_color = colors.get(vital_sign, 'red')  # Default color is red
        ax.plot(patient_data['obs_ts'], patient_data[vital_sign], marker='o', linestyle='-', color=vital_color, label=vital_sign)
        
        # Plot threshold lines
        lower_threshold, upper_threshold = thresholds.get(vital_sign, (None, None))
        if lower_threshold is not None:
            ax.axhline(y=lower_threshold, color='r', linestyle='--')
        if upper_threshold is not None:
            ax.axhline(y=upper_threshold, color='r', linestyle='--')
        
        # Set y-axis labels for each vital sign
        if vital_sign == 'T':
            ax.set_ylabel('Temperature (°C)')
        elif vital_sign == 'HR':
            ax.set_ylabel('Heart Rate')
        elif vital_sign == 'SBP':
            ax.set_ylabel('Systolic Blood Pressure (mmHg)')
        elif vital_sign == 'R':
            ax.set_ylabel('Respiratory Rate (beats/min)')
        
        # Check if vital sign is outside of the threshold range
        lower_threshold_alert = (patient_data[vital_sign] < lower_threshold).any() if lower_threshold else False
        upper_threshold_alert = (patient_data[vital_sign] > upper_threshold).any() if upper_threshold else False

        # Get the x-coordinate for adding the alert message
        alert_x_position = patient_data['obs_ts'].iloc[0]  # Assuming obs_ts is the column containing timestamps
        
        # Add "Sepsis Alert!" message if vital sign is outside of the threshold range
        if lower_threshold_alert:
            alert_y_position = lower_threshold - 0.5  # Position the message just below the lower threshold
            ax.text(alert_x_position, alert_y_position, 'Sepsis Alert!', color='red', fontsize=8,
                    verticalalignment='center', rotation='vertical')  # Set rotation to 'vertical'

        if upper_threshold_alert:
            alert_y_position = upper_threshold + 0.5  # Position the message just above the upper threshold
            ax.text(alert_x_position, alert_y_position, 'Sepsis Alert!', color='red', fontsize=8,
                    verticalalignment='center', rotation='vertical')  # Set rotation to 'vertical'

    # Set x-axis label only for the bottom subplot
    axs[-1].set_xlabel('Date and Time')
    
    # Set a title for the entire plot (general plot for each patient)
    plt.suptitle(f'Patient {patient_id} - Vital Sign Trends')
    
    # Add a legend for each subplot
    for ax in axs:
        ax.legend(loc='upper right', fontsize='small')
    
    # Adjust layout and show the plot for the current patient
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Add space at the top for the suptitle
    plt.show()
